<a href="https://colab.research.google.com/github/Satyendra0207/Spark_collab/blob/main/Satyendra_Singh(2132)_Regression_assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 52.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=da8aca722f2ebdd3a9dfc466fe63813476060079efa5efdd9f22937e06d3d3c0
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
session=SparkSession.builder.appName("regression_project").getOrCreate()

In [6]:
#importing data set 
data=session.read.csv("automobile2.csv",header=True,inferSchema=True)
data.show(5)

+---+-----------+---------+------------+-----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+-----------------+----------+--------+-----------+-----+
|_c0|       make|fuel_type|num_of_doors| body_style|drive_wheels|engine_location|wheel_base|length|width|height|curb_weight|engine_type|num_of_cylinders|engine_size|fuel_system|compression_ratio|horsepower|city_mpg|highway_mpg|price|
+---+-----------+---------+------------+-----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+-----------------+----------+--------+-----------+-----+
|  0|alfa-romero|      gas|         two|convertible|         rwd|          front|      88.6| 168.8| 64.1|  48.8|       2548|       dohc|            four|        130|       mpfi|              9.0|       111|      21|         27|13495|
|  1|alfa-romero|      gas|         two|convertible|         rwd

In [7]:
data.columns

['_c0',
 'make',
 'fuel_type',
 'num_of_doors',
 'body_style',
 'drive_wheels',
 'engine_location',
 'wheel_base',
 'length',
 'width',
 'height',
 'curb_weight',
 'engine_type',
 'num_of_cylinders',
 'engine_size',
 'fuel_system',
 'compression_ratio',
 'horsepower',
 'city_mpg',
 'highway_mpg',
 'price']

In [8]:
#checking data types 
data.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- make: string (nullable = true)
 |-- fuel_type: string (nullable = true)
 |-- num_of_doors: string (nullable = true)
 |-- body_style: string (nullable = true)
 |-- drive_wheels: string (nullable = true)
 |-- engine_location: string (nullable = true)
 |-- wheel_base: double (nullable = true)
 |-- length: double (nullable = true)
 |-- width: double (nullable = true)
 |-- height: double (nullable = true)
 |-- curb_weight: integer (nullable = true)
 |-- engine_type: string (nullable = true)
 |-- num_of_cylinders: string (nullable = true)
 |-- engine_size: integer (nullable = true)
 |-- fuel_system: string (nullable = true)
 |-- compression_ratio: double (nullable = true)
 |-- horsepower: integer (nullable = true)
 |-- city_mpg: integer (nullable = true)
 |-- highway_mpg: integer (nullable = true)
 |-- price: integer (nullable = true)



In [85]:
#importing required functions 
from pyspark.ml.feature import VectorAssembler,StringIndexer,OneHotEncoder

##Changing string datatype of attruibutes using StringIndexer
str_idx=StringIndexer(inputCols=['make','fuel_type','num_of_doors','body_style',"num_of_cylinders","drive_wheels"],outputCols=['make_2','fuel_type_2','num_of_doors_2','body_style_2',"num_of_cylinders2","drive_wheels2"],handleInvalid="skip",stringOrderType="frequencyDesc")

In [86]:
#assembling features using Vector Assembler
#vec_assem=VectorAssembler(inputCols=["new_make","new_fuel_type","new_num_of_doors2","new_num_of_cylinders"],outputCol="allfeatures")
vec_assem=VectorAssembler(inputCols=['make_2','fuel_type_2','num_of_doors_2','body_style_2',"num_of_cylinders2","drive_wheels2","wheel_base",'length','width','height','curb_weight','engine_size','horsepower','city_mpg','highway_mpg'],outputCol="allfeatures")

In [87]:
#importing model 
from pyspark.ml.regression import LinearRegression
lr=LinearRegression(featuresCol="allfeatures",labelCol="price")

In [88]:
from pyspark.ml import Pipeline
mypipe=Pipeline(stages=[str_idx,vec_assem,lr])

In [89]:
training,test=data.randomSplit([0.8,0.2])

In [90]:
training.show(5)

+---+-----------+---------+------------+-----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+-----------------+----------+--------+-----------+-----+
|_c0|       make|fuel_type|num_of_doors| body_style|drive_wheels|engine_location|wheel_base|length|width|height|curb_weight|engine_type|num_of_cylinders|engine_size|fuel_system|compression_ratio|horsepower|city_mpg|highway_mpg|price|
+---+-----------+---------+------------+-----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+-----------------+----------+--------+-----------+-----+
|  0|alfa-romero|      gas|         two|convertible|         rwd|          front|      88.6| 168.8| 64.1|  48.8|       2548|       dohc|            four|        130|       mpfi|              9.0|       111|      21|         27|13495|
|  1|alfa-romero|      gas|         two|convertible|         rwd

In [91]:
fitmodel=mypipe.fit(training)
result = fitmodel.transform(test)
result.show(5)

+---+----+---------+------------+----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+-----------------+----------+--------+-----------+-----+------+-----------+--------------+------------+-----------------+-------------+--------------------+------------------+
|_c0|make|fuel_type|num_of_doors|body_style|drive_wheels|engine_location|wheel_base|length|width|height|curb_weight|engine_type|num_of_cylinders|engine_size|fuel_system|compression_ratio|horsepower|city_mpg|highway_mpg|price|make_2|fuel_type_2|num_of_doors_2|body_style_2|num_of_cylinders2|drive_wheels2|         allfeatures|        prediction|
+---+----+---------+------------+----------+------------+---------------+----------+------+-----+------+-----------+-----------+----------------+-----------+-----------+-----------------+----------+--------+-----------+-----+------+-----------+--------------+------------+-----------------+-------------+------

In [92]:
from pyspark.ml.evaluation import RegressionEvaluator
eval_obj=RegressionEvaluator(predictionCol="prediction",labelCol="price",metricName="r2")
accuracy=eval_obj.evaluate(result)
print(accuracy)

0.8085450582623148


In [93]:
metric_l=["rmse","mse","mae","r2"]
for metric in metric_l:
  reg_eval=RegressionEvaluator(predictionCol="prediction",labelCol="price",metricName=metric)
  accuracy=reg_eval.evaluate(result)
  print(metric,":",accuracy)


rmse : 3944.8165139125504
mse : 15561577.328437168
mae : 2547.248795292363
r2 : 0.8085450582623148
